In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

# Import our input dataset
diabetes_df = pd.read_csv('diabetes.csv')
diabetes_df.head()

C:\Users\otrin\.conda\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\otrin\.conda\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\otrin\.conda\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\otrin\.conda\envs\PythonData\lib\site-packages\tensorflow\

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


This dataset is from the National Institute of Diabetes and Digestive and Kidney Diseases (NIDDK) and contains the patient information of 786 women. It is used as a real-world practice dataset to build a predictive diagnostic model. Since there are only 786 data points across eight features, this dataset is well suited for a logistic regression model, but still large enough to build a neural network model. Now that we have our dataset loaded into Pandas, we need to prepare the data to train both models. With our logistic regression model, there is no preprocessing or scaling required for the data. However, our basic neural network needs the numerical variables standardized. Therefore, we’ll need to keep track of a scaled and unscaled training dataset such that both models have the correct input data in their preferred formats. To split the data, we need to add and run the following code:

In [2]:
# Remove diabetes outcome target from features data
y = diabetes_df.Outcome
X = diabetes_df.drop(columns="Outcome")

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [3]:
# Preprocess numerical data for neural network

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Logistic regression models can be built using Scikit-learn’s LogisticRegression class in the linear_model module.
For our purposes, we’ll use our basic logistic regression parameters, which include:

- The solver parameter is set to 'lbfgs', which is an algorithm for learning and optimization. The particular solver isn’t very important in this example, but note that a number of optimizers exist.
- The max_iter parameter will be set to 200 iterations, which will give the model sufficient opportunity to converge on effective weights

Putting all of our arguments together, we’ll add and run the following code in the notebook:

In [4]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.729


Next, we need to build, compile, and evaluate our basic neural network model. Again, we’ll use our typical binary classifier parameters:

- Our single hidden layer will have an input_dim equal to 8, 16 neuron units, and will use the relu activation function.
- The loss function should be binary_crossentropy, using the adam optimizer.
- Our model should provide the additional accuracy scoring metric and train over a maximum of 50 epochs.

__NOTE__
Compared to the 200 training iterations for our logistic regression model, we’ll only train our neural network model through 50 epochs—this will limit the risk of overfitting our model.

Again, we need to add and run the following code in our notebooks:

In [5]:
# Define the basic neural network model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=8))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
576/576 [==============================] - 0s 134us/sample - loss: 0.8460 - acc: 0.3872
Epoch 2/100
576/576 [==============================] - 0s 19us/sample - loss: 0.7779 - acc: 0.4444
Epoch 3/100
576/576 [==============================] - 0s 17us/sample - loss: 0.7210 - acc: 0.5156
Epoch 4/100
576/576 [==============================] - 0s 17us/sample - loss: 0.6780 - acc: 0.6007
Epoch 5/100
576/576 [==============================] - 0s 17us/sample - loss: 0.6407 - acc: 0.6562
Epoch 6/100
576/576 [==============================] - 0s 17us/sample - loss: 0.6115 - acc: 0.6979
Epoch 7/100
576/576 [==============================] - 0s 17us/sample - loss: 0.5889 - acc: 0.7222
Epoch 8/100
576/576 [==============================] - 0s 17us/sample - loss: 0.5698 - acc: 0.

576/576 [==============================] - 0s 17us/sample - loss: 0.4270 - acc: 0.7951
Epoch 76/100
576/576 [==============================] - 0s 16us/sample - loss: 0.4267 - acc: 0.7969
Epoch 77/100
576/576 [==============================] - 0s 16us/sample - loss: 0.4262 - acc: 0.7969
Epoch 78/100
576/576 [==============================] - 0s 16us/sample - loss: 0.4258 - acc: 0.7969
Epoch 79/100
576/576 [==============================] - 0s 17us/sample - loss: 0.4255 - acc: 0.7951
Epoch 80/100
576/576 [==============================] - 0s 17us/sample - loss: 0.4254 - acc: 0.7951
Epoch 81/100
576/576 [==============================] - 0s 17us/sample - loss: 0.4250 - acc: 0.7951
Epoch 82/100
576/576 [==============================] - 0s 17us/sample - loss: 0.4248 - acc: 0.7951
Epoch 83/100
576/576 [==============================] - 0s 17us/sample - loss: 0.4245 - acc: 0.7917
Epoch 84/100
576/576 [==============================] - 0s 17us/sample - loss: 0.4241 - acc: 0.7934
Epoch 85/100


If we compare both model’s predictive accuracy, their output is very similar. Either model was able to predict whether or not a patient is diagnosed with diabetes more than 70% of the time.

Beyond the performance of both models, there are a few other factors to consider when selecting a model for your data. First, neural networks are prone to overfitting and can be more difficult to train than a straightforward logistic regression model. Therefore, if you are trying to build a classifier with limited data points (typically fewer than a thousand data points), or if your dataset has only a few features, neural networks may be overcomplicated. Additionally, logistic regression models are easier to dissect and interpret than their neural network counterparts, which tends to put more traditional data scientists and non-data experts at ease. In contrast, neural networks (and especially deep neural networks) thrive in large datasets. Datasets with thousands of data points, or datasets with complex features, may overwhelm the logistic regression model, while a deep learning model can evaluate every interaction within and across neurons. Therefore, the decision between using a logistic regression model and basic neural network model is nuanced and, in most cases, a matter of preference for the data scientist.

